In [ ]:
import time
import pandas as pd
import threading
import ccxt
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Initialize the KuCoin and Binance clients
kucoin = ccxt.kucoin()
binance = ccxt.binance()
bitget = ccxt.bitget()  # Initialize Bitget client

# Firebase initialization
cred = credentials.Certificate('crypto-9657a-firebase-adminsdk-jpj5s-9afd4a7616.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

# List of coins to fetch data for
coins = ["sxp", "chess", "blz", "joe", "perl", "ach", "gmt", "xrp", "akro", "zil", "cfx", "adx", "chz", "bel", "alpaca", "elf", "epx", "pros", "t", "dar", "agix", "mob", "id", "trx", "key", "tru", "amb", "magic", "lina", "lever", "btc", "eth", "tomo", "dodo", "cvp", "data", "ata", "cos", "fida", "fis", "loom", "super", "pepe", "matic", "ada", "doge", "mav", "xec", "sui", "eos", "ftm", "xlm"]

# Function to check if coin is supported on an exchange
def is_coin_supported(exchange, coin):
    markets = exchange.load_markets()
    return f"{coin.upper()}/USDT" in markets

# Fetch and store recent trades
def fetch_and_store_recent_trades(exchange, coin):
    last_trade_timestamp = None
    while True:
        try:
            trades = exchange.fetch_trades(f'{coin.upper()}/USDT')
            for trade in trades:
                trade_timestamp = pd.to_datetime(trade['timestamp'], unit='ms')
                if last_trade_timestamp is None or trade_timestamp > last_trade_timestamp:
                    doc_ref = db.collection(f'{coin}usdt').document()
                    doc_ref.set({
                        'timestamp': trade_timestamp,
                        'price': trade['price'],
                        'volume': trade['amount']
                    })
                    last_trade_timestamp = trade_timestamp
        except Exception as e:
            print(f'Error fetching and storing trades for {coin} from {exchange.name}: {e}')
            time.sleep(30)
            continue
        time.sleep(5)

# Function to delete data older than 7 Days
def delete_old_data(coin):
    while True:
        try:
            col_ref = db.collection(f'{coin}usdt')
            docs = col_ref.stream()

            for doc in docs:
                if doc.to_dict()['timestamp'] < pd.Timestamp.now() - pd.DateOffset(days=7):
                    doc.reference.delete()
        except Exception as e:
            print(f'Error deleting old data for {coin}: {e}')

        time.sleep(21600)  # Sleep for 6 hours

# Start threads for each coin and exchange
def start_threads():
    for coin in coins:
        # Check for coin support in Binance
        if is_coin_supported(binance, coin):
            threading.Thread(target=fetch_and_store_recent_trades, args=(binance, coin)).start()
            threading.Thread(target=delete_old_data, args=(coin,)).start()

        # Check for coin support in KuCoin
        if is_coin_supported(kucoin, coin):
            threading.Thread(target=fetch_and_store_recent_trades, args=(kucoin, coin)).start()
            threading.Thread(target=delete_old_data, args=(coin,)).start()

        # Check for coin support in Bitget
        if is_coin_supported(bitget, coin):  # Add this block for Bitget
            threading.Thread(target=fetch_and_store_recent_trades, args=(bitget, coin)).start()
            threading.Thread(target=delete_old_data, args=(coin,)).start()

# Start threads
start_threads()

# Keep the script running
while True:
    time.sleep(30)
